In [2]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [3]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
print(game)
print(game.board_value())

+------+------+------+------+
|  2   |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
4


In [4]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

In [12]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [13]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

0
+------+------+------+------+------+
|  2   |  32  |  2   |  32  |  4   |
+------+------+------+------+------+
| 128  |  8   |  16  | 256  |  2   |
+------+------+------+------+------+
|  2   | 512  |  2   |  64  |  4   |
+------+------+------+------+------+
|  16  |  2   | 1024 |  32  | 512  |
+------+------+------+------+------+
|  2   |  4   | 128  |  2   |  32  |
+------+------+------+------+------+
2
1680016.0
275794686345.4721


KeyboardInterrupt: 

In [19]:
f = open("res.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

In [6]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [7]:
 time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+------+
| 1024 |      | 1024 |      | 1024 |
+------+------+------+------+------+
|      |      |      |  2   | 2048 |
+------+------+------+------+------+
|      | 1024 |  4   |  4   | 256  |
+------+------+------+------+------+
| 256  |  64  |  32  |  16  |  4   |
+------+------+------+------+------+
|  2   |  16  |  4   | 128  |  4   |
+------+------+------+------+------+
1
8541784.0
78311873305.57367
1
+------+------+------+------+------+
|      |  2   |  8   |  8   |  2   |
+------+------+------+------+------+
|      |  16  |  64  |  16  | 1024 |
+------+------+------+------+------+
|      |      |  64  |  32  | 2048 |
+------+------+------+------+------+
|      |  8   |  4   | 128  |  2   |
+------+------+------+------+------+
|  2   |      |  2   |      |  2   |
+------+------+------+------+------+
1
5269224.0
52775393004.06262
2
+------+------+------+------+------+
|      |      |      |  2   |  32  |
+------+------+------+------+------+
|      | 

19
+------+------+------+------+------+
|      |      |  4   |      |      |
+------+------+------+------+------+
|      |  4   |  8   | 1024 |      |
+------+------+------+------+------+
|  4   |  64  |  16  |      |      |
+------+------+------+------+------+
|  2   |  2   |  2   | 1024 |  2   |
+------+------+------+------+------+
| 128  |  16  |  64  | 2048 |  64  |
+------+------+------+------+------+
1
6320768.0
136194590894.04822
20
+------+------+------+------+------+
|  2   |      |  2   |  4   |  16  |
+------+------+------+------+------+
|      |      |      | 2048 |  2   |
+------+------+------+------+------+
| 1024 |  4   |  16  |  32  |  8   |
+------+------+------+------+------+
|  2   |  4   |  2   |  8   | 256  |
+------+------+------+------+------+
| 1024 |  8   | 1024 |  16  |  2   |
+------+------+------+------+------+
1
7407624.0
209399185519.77643
21
+------+------+------+------+------+
|      | 1024 |      | 1024 | 2048 |
+------+------+------+------+------+
|  2

38
+------+------+------+------+------+
|  4   |  64  |  16  |  32  |  16  |
+------+------+------+------+------+
|  2   |  8   | 2048 |      |      |
+------+------+------+------+------+
| 256  |  32  |  8   |      |      |
+------+------+------+------+------+
|  2   |      |  2   |      |  2   |
+------+------+------+------+------+
|  2   |  8   |      |      |      |
+------+------+------+------+------+
1
4266724.0
74907041632.48935
39
+------+------+------+------+------+
|      |      |  2   | 512  | 2048 |
+------+------+------+------+------+
|      |  4   |  16  |  8   |  2   |
+------+------+------+------+------+
|      |  4   |  8   | 128  |  4   |
+------+------+------+------+------+
|  8   |  32  |  4   |  16  |  8   |
+------+------+------+------+------+
|      |      |  4   | 128  |  8   |
+------+------+------+------+------+
1
4491160.0
108955697315.07237
40
+------+------+------+------+------+
|      |      |  8   |  2   |  4   |
+------+------+------+------+------+
|    

57
+------+------+------+------+------+
| 2048 |  8   | 256  |  2   |      |
+------+------+------+------+------+
|  32  | 128  |  4   |  2   |  4   |
+------+------+------+------+------+
|  8   |  16  |  2   |  8   |      |
+------+------+------+------+------+
| 1024 |  64  |  4   |  64  |      |
+------+------+------+------+------+
|  4   |  2   |  8   |      |      |
+------+------+------+------+------+
1
5334608.0
136194639236.4128
58
+------+------+------+------+------+
|      |  16  |      |  32  |  64  |
+------+------+------+------+------+
|      |      |      |  8   |  8   |
+------+------+------+------+------+
| 2048 |  16  |  4   |  16  |  2   |
+------+------+------+------+------+
| 256  |  32  |  4   | 128  |  32  |
+------+------+------+------+------+
|  2   |  4   | 256  |  16  |  2   |
+------+------+------+------+------+
1
4350140.0
127682475128.38283
59
+------+------+------+------+------+
|  4   |  2   |  16  |  16  |  4   |
+------+------+------+------+------+
| 128

76
+------+------+------+------+------+
|      |  2   |  32  |  8   |  8   |
+------+------+------+------+------+
|      |  16  | 2048 |  64  |  2   |
+------+------+------+------+------+
|      |      |  4   |  16  |  2   |
+------+------+------+------+------+
|      |      |  4   |  64  |  2   |
+------+------+------+------+------+
|  16  |      |  16  |      |  16  |
+------+------+------+------+------+
1
4204976.0
253662563654.04202
77
+------+------+------+------+------+
|  16  |  64  |  2   |      |      |
+------+------+------+------+------+
| 128  |  2   |  4   |  2   |      |
+------+------+------+------+------+
|  4   |  16  | 512  |  8   | 2048 |
+------+------+------+------+------+
|  32  | 128  |  8   |  2   | 512  |
+------+------+------+------+------+
|  4   |  8   |  2   |  4   |  2   |
+------+------+------+------+------+
1
4757272.0
187267667571.72858
78
+------+------+------+------+------+
|      |  2   |  64  |  8   |  16  |
+------+------+------+------+------+
|   

95
+------+------+------+------+------+
| 2048 |  2   | 512  |      |      |
+------+------+------+------+------+
|  2   |  64  |  2   |      |      |
+------+------+------+------+------+
| 256  |  2   |  64  |  2   |      |
+------+------+------+------+------+
|  2   |  64  |  8   | 256  |  4   |
+------+------+------+------+------+
|  16  | 256  |  4   |  8   |  2   |
+------+------+------+------+------+
1
4665788.0
125980090166.755
96
+------+------+------+------+------+
|      |      | 256  |      | 256  |
+------+------+------+------+------+
|      | 256  |  2   |      | 1024 |
+------+------+------+------+------+
|      |  4   |  8   |  4   |  2   |
+------+------+------+------+------+
|  4   |  64  |  32  |  64  |  64  |
+------+------+------+------+------+
|  2   |  4   |  8   | 2048 |  2   |
+------+------+------+------+------+
1
5453008.0
204292039877.92184
97
+------+------+------+------+------+
| 128  |  2   |  16  |  8   | 512  |
+------+------+------+------+------+
|  4  

In [9]:
f = open("res_boot2.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 3 - Rand Walk

In [10]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            board.playRandUser()
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

256.0
+------+------+------+------+------+
|  4   |  2   |  4   |  8   |  2   |
+------+------+------+------+------+
|  2   |  4   |  8   | 128  |  8   |
+------+------+------+------+------+
|  4   |  8   |  32  |  8   |  2   |
+------+------+------+------+------+
|  8   |  64  | 256  |  16  |  8   |
+------+------+------+------+------+
|  16  |  4   |  2   | 128  |  2   |
+------+------+------+------+------+
2
104488.0
0
512.0
+------+------+------+------+------+
|  32  |  2   |  8   |  4   |  2   |
+------+------+------+------+------+
|  4   | 128  | 256  |  32  |  16  |
+------+------+------+------+------+
|  2   | 512  |  64  |  16  |  8   |
+------+------+------+------+------+
| 128  |  2   |  32  |  2   |  4   |
+------+------+------+------+------+
|  2   |  4   |  8   |  4   |  2   |
+------+------+------+------+------+
2
368428.0
0
128.0
+------+------+------+------+------+
|  4   |  2   |  64  |  8   |  4   |
+------+------+------+------+------+
|  16  |  4   |  32  |  4   |  

1024.0
+------+------+------+------+------+
|  2   |  16  |  2   |  8   |  2   |
+------+------+------+------+------+
| 512  | 1024 |  32  | 512  |  4   |
+------+------+------+------+------+
|  2   |  16  | 128  |  16  |  2   |
+------+------+------+------+------+
|  16  |  8   |  64  |  32  |  16  |
+------+------+------+------+------+
|  4   |  16  |  2   |  4   |  2   |
+------+------+------+------+------+
2
1597132.0
0
256.0
+------+------+------+------+------+
|  2   |  4   |  32  |  4   |  2   |
+------+------+------+------+------+
|  4   |  16  |  8   |  2   | 128  |
+------+------+------+------+------+
|  16  |  64  |  4   |  64  |  2   |
+------+------+------+------+------+
| 256  |  4   |  16  |  2   |  4   |
+------+------+------+------+------+
|  2   |  16  |  2   |  4   |  2   |
+------+------+------+------+------+
2
92368.0
0
1024.0
+------+------+------+------+------+
|  2   |  4   |  8   |  16  |  2   |
+------+------+------+------+------+
|  4   |  2   |  4   |  64  |

512.0
+------+------+------+------+------+
|  2   |  16  |  4   |  2   |  4   |
+------+------+------+------+------+
|  8   |  4   | 128  |  8   |  2   |
+------+------+------+------+------+
|  4   | 512  |  16  |  32  |  8   |
+------+------+------+------+------+
|  32  | 128  |  4   |  8   |  4   |
+------+------+------+------+------+
|  2   |  4   |  2   |  16  |  2   |
+------+------+------+------+------+
2
298120.0
0
256.0
+------+------+------+------+------+
|  2   |  8   |  16  |  4   |  2   |
+------+------+------+------+------+
|  16  |  32  |  8   |  16  |  8   |
+------+------+------+------+------+
|  4   |  16  | 256  |  8   |  32  |
+------+------+------+------+------+
|  8   |  64  |  8   |  2   |  4   |
+------+------+------+------+------+
|  16  |  8   |  4   |  16  |  2   |
+------+------+------+------+------+
2
73744.0
0
1024.0
+------+------+------+------+------+
|  4   |  2   |  4   |  8   |  2   |
+------+------+------+------+------+
|  8   | 1024 |  32  | 128  |  

256.0
+------+------+------+------+------+
|  8   |  32  |  8   |  2   |  4   |
+------+------+------+------+------+
|  2   | 256  |  2   |  64  |  2   |
+------+------+------+------+------+
|  8   |  32  |  8   |  32  |  8   |
+------+------+------+------+------+
|  4   |  16  |  32  |  8   |  2   |
+------+------+------+------+------+
|  32  |  64  |  2   |  32  |  4   |
+------+------+------+------+------+
2
80584.0
0
512.0
+------+------+------+------+------+
|  2   |  64  |  4   |  2   |  4   |
+------+------+------+------+------+
|  16  |  32  |  8   |  4   |  8   |
+------+------+------+------+------+
|  2   |  8   | 128  |  32  |  2   |
+------+------+------+------+------+
|  16  |  4   |  8   | 512  |  32  |
+------+------+------+------+------+
|  2   |  64  |  4   |  8   |  2   |
+------+------+------+------+------+
2
290728.0
0
128.0
+------+------+------+------+------+
|  4   |  2   |  8   |  2   |  4   |
+------+------+------+------+------+
|  2   |  8   |  32  |  8   |  1

128.0
+------+------+------+------+------+
|  2   |  4   |  2   |  8   |  4   |
+------+------+------+------+------+
|  8   |  64  | 128  |  16  |  8   |
+------+------+------+------+------+
|  32  | 128  |  32  |  8   |  2   |
+------+------+------+------+------+
|  2   |  16  |  8   |  64  |  4   |
+------+------+------+------+------+
|  8   |  32  |  4   |  2   |  32  |
+------+------+------+------+------+
2
46036.0
0
256.0
+------+------+------+------+------+
|  2   |  32  |  8   |  16  |  8   |
+------+------+------+------+------+
|  4   |  16  | 256  |  8   |  16  |
+------+------+------+------+------+
|  32  |  8   |  32  |  4   |  2   |
+------+------+------+------+------+
|  8   |  4   |  16  |  8   |  4   |
+------+------+------+------+------+
|  4   |  32  |  8   |  2   |  32  |
+------+------+------+------+------+
2
72220.0
0
256.0
+------+------+------+------+------+
|  8   |  2   |  32  |  16  |  4   |
+------+------+------+------+------+
|  64  |  8   |  64  | 256  |  2 

In [11]:
f = open("res_rand5-score.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 4 - How High n-step

In [14]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
                
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        #print(self.unexplored_moves)
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        #print(leaf.grid)
        sim_res = rollout_o(leaf, start_grid)
        #print(sim_res)
        backpropagate(leaf, sim_res)
        #print(root.children)

    # When done iterating, return the 'best' child of the root node.\
    try:
        return best_child(root).move
    except:
        #print(root.children)
        return root

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        #print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)
    
    # If the node is terminal, return it
    if node.is_terminal_state:
        #print(1)
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    return sample.board_value()/(16384 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [15]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board,100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

131072.0
+------+------+------+------+------+
|  8   | 128  |  16  |  8   |  4   |
+------+------+------+------+------+
|  2   |  16  |  64  |  4   |  32  |
+------+------+------+------+------+
|32768 |  8   | 1024 | 256  |131072|
+------+------+------+------+------+
|  16  |  32  | 128  |  32  |  8   |
+------+------+------+------+------+
|  2   |  4   |  2   |  8   |  4   |
+------+------+------+------+------+
2
18254766220.0
338785736864.6967
1024.0
+------+------+------+------+------+
|  2   | 256  |  2   |  64  |  2   |
+------+------+------+------+------+
|  4   |  16  |  4   |  2   |  4   |
+------+------+------+------+------+
| 256  |  2   |  16  | 1024 |  32  |
+------+------+------+------+------+
|  4   |  64  |  4   |  32  |  4   |
+------+------+------+------+------+
|  8   | 256  |  2   |  16  |  2   |
+------+------+------+------+------+
2
1256380.0
73204960540.40031
1024.0
+------+------+------+------+------+
|  4   |  8   |  4   |  16  |  2   |
+------+------+------+---

KeyboardInterrupt: 